# 线性回归的简洁实现
通过使用深度学习框架来简洁地实现线性回归模型生成数据集

## 3.3.1. 生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data   # 其中包含有一些数据处理的模块
from d2l import torch as d2l


true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 3.3.2. 读取数据集
调用框架中现有的API来读取数据

In [2]:
# 将features和labels作为API的参数传递，并在实例化数据迭代器对象时指定batch_size
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器。"""
    # TensorDataset对给定的tensor数据(样本和标签)，将它们包装成dataset
    dataset = data.TensorDataset(*data_arrays)  
    # DataLoader数据加载器，组合数据集和采样器，并在数据集上提供单进程或多进程迭代器。它可以对我们上面所说的数据集Dataset作进一步的设置
    return data.DataLoader(dataset, batch_size, shuffle=is_train)   

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [3]:
next(iter(data_iter))

[tensor([[-4.6024e-01,  3.2508e-01],
         [ 2.1815e-03, -5.5393e-01],
         [-3.5364e-01, -2.1935e+00],
         [ 1.7869e+00, -6.4668e-01],
         [ 1.6216e+00,  5.6416e-02],
         [ 4.8055e-01,  1.3658e+00],
         [-5.5038e-02,  8.0333e-01],
         [-1.8361e-01, -3.5865e-01],
         [ 1.8611e-01,  5.6090e-01],
         [ 1.3797e+00,  8.5556e-01]]),
 tensor([[ 2.1662],
         [ 6.0961],
         [10.9311],
         [ 9.9692],
         [ 7.2594],
         [ 0.5321],
         [ 1.3449],
         [ 5.0531],
         [ 2.6628],
         [ 4.0254]])]

## 3.3.3. 定义模型
使用框架的预定义好的层

In [4]:
# `nn` 是神经网络的缩写
from torch import nn

# 首先定义一个模型变量net，它是一个Sequential类的实例
# Sequential类为串联在一起的多个层定义了一个容器
# nn.Sequential是一个容器，模块将按照它们在构造函数中传递的顺序添加到它
# 将两个参数传递到nn.Linear中。第一个指定输入特征形状，第二个指定输出特征形状
net = nn.Sequential(nn.Linear(2, 1))

## 3.3.4. 初始化模型参数

In [5]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 3.3.5. 定义损失函数
MSELoss类，也称为平方L2范数。默认情况下，它返回所有样本损失的平均值。

In [6]:
loss = nn.MSELoss()

## 3.3.6. 定义优化算法

In [7]:
# 实例化SGD实例时要指定优化的参数（可通过net.parameters()从模型中获得）以及优化算法所需的超参数字典。
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 3.3.7. 训练

In [8]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()   # 这里pytorch已经做了sum
        trainer.step()   # 进行一次模型更新
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000227
epoch 2, loss 0.000094
epoch 3, loss 0.000094
